# การจดจำชื่อเฉพาะ (Named Entity Recognition - NER)

สมุดบันทึกนี้มาจาก [AI for Beginners Curriculum](http://aka.ms/ai-beginners)

ในตัวอย่างนี้ เราจะเรียนรู้วิธีการฝึกโมเดล NER บนชุดข้อมูล [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) จาก Kaggle ก่อนเริ่มต้น กรุณาดาวน์โหลดไฟล์ [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) ลงในไดเรกทอรีปัจจุบัน


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## การเตรียมชุดข้อมูล

เราจะเริ่มต้นด้วยการอ่านชุดข้อมูลเข้าสู่ dataframe หากคุณต้องการเรียนรู้เพิ่มเติมเกี่ยวกับการใช้ Pandas สามารถเยี่ยมชม [บทเรียนเกี่ยวกับการประมวลผลข้อมูล](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) ใน [Data Science for Beginners](http://aka.ms/datascience-beginners)


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

ตอนนี้เราจำเป็นต้องทำแบบเดียวกันกับคำศัพท์ เพื่อความง่าย เราจะสร้างคำศัพท์โดยไม่คำนึงถึงความถี่ของคำ ในชีวิตจริงคุณอาจต้องการใช้ Keras vectorizer และจำกัดจำนวนคำ


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

เราจำเป็นต้องสร้างชุดข้อมูลของประโยคสำหรับการฝึกอบรม มาลูปผ่านชุดข้อมูลต้นฉบับและแยกประโยคแต่ละประโยคออกเป็น `X` (รายการของคำ) และ `Y` (รายการของโทเค็น):


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

เพื่อความง่าย เราจะเติมประโยคทั้งหมดด้วย 0 โทเค็นจนถึงความยาวสูงสุด ในชีวิตจริง เราอาจต้องการใช้กลยุทธ์ที่ฉลาดกว่า และเติมลำดับเฉพาะภายในหนึ่งมินิแบตช์เท่านั้น


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## การกำหนดเครือข่ายการจัดประเภทโทเค็น

เราจะใช้เครือข่าย LSTM แบบสองชั้นที่มีการทำงานแบบสองทิศทางสำหรับการจัดประเภทโทเค็น เพื่อที่จะใช้ตัวจำแนกแบบหนาแน่นกับผลลัพธ์แต่ละตัวของชั้น LSTM สุดท้าย เราจะใช้โครงสร้าง `TimeDistributed` ซึ่งจะทำการจำลองชั้นหนาแน่นเดียวกันไปยังผลลัพธ์แต่ละตัวของ LSTM ในแต่ละขั้นตอน:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

โปรดทราบว่าเรากำลังระบุ `maxlen` สำหรับชุดข้อมูลของเรา - ในกรณีที่เราต้องการให้เครือข่ายสามารถจัดการกับลำดับที่มีความยาวแปรผันได้ เราจำเป็นต้องมีความชาญฉลาดมากขึ้นเมื่อกำหนดเครือข่าย

ตอนนี้เรามาเริ่มฝึกโมเดลกัน สำหรับความรวดเร็ว เราจะฝึกเพียงหนึ่ง epoch เท่านั้น แต่คุณสามารถลองฝึกเป็นเวลานานขึ้นได้ นอกจากนี้ คุณอาจต้องการแยกบางส่วนของชุดข้อมูลออกเป็นชุดข้อมูลการฝึก เพื่อสังเกตความแม่นยำในการตรวจสอบ


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## การทดสอบผลลัพธ์

มาลองดูว่าระบบการจดจำเอนทิตีของเราทำงานอย่างไรกับประโยคตัวอย่าง:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## ข้อสรุป

แม้แต่โมเดล LSTM แบบง่าย ๆ ก็สามารถให้ผลลัพธ์ที่น่าพอใจในงาน NER ได้ อย่างไรก็ตาม หากต้องการผลลัพธ์ที่ดียิ่งขึ้น คุณอาจต้องใช้โมเดลภาษาขนาดใหญ่ที่ผ่านการฝึกมาแล้ว เช่น BERT การฝึก BERT สำหรับ NER โดยใช้ไลบรารี Huggingface Transformers ได้อธิบายไว้ [ที่นี่](https://huggingface.co/course/chapter7/2?fw=pt)



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้องมากที่สุด แต่โปรดทราบว่าการแปลโดยอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามืออาชีพ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
